## AI Keyboard

In [6]:
#Imorting libraries
import cv2
import cvzone
from cvzone.HandTrackingModule import HandDetector
from selenium import webdriver
from time import sleep
import warnings
warnings.filterwarnings("ignore")

#Calling hand detector class to fetch hand loaction and landmarks\
detector = HandDetector(detectionCon=0.8)

#Defining keys
keys = [["Q","W","E","R","T","Y","U","I","O","P"],
      ["A","S","D","F","G","H","J","K","L","<"],
      ["Z","X","C","V","B","N","M",","," ",">"]]

#Opening web driver
driver_path = "D:\\Learning\\College Section\\Setup\\Drivers\\chromedriver.exe"
brave_path = "C:\\Program Files\\BraveSoftware\\Brave-Browser\\Application\\brave.exe"
option = webdriver.ChromeOptions()
option.binary_location = brave_path
browser = webdriver.Chrome(executable_path=driver_path, chrome_options=option)\

#Button class which will hold button pos, size and text 
class Button:
    def __init__(self,pos,text,size=[50,50]):
        self.pos = tuple(pos)
        self.size = tuple(size)
        self.text = text

#function which will draw text and rectangle on the camera
def drawAll(img,buttonList):
    for button in buttonList:
        x,y = button.pos
        w,h = button.size
        cv2.rectangle(img,button.pos,(x+w,y+h),(255,0,255),cv2.FILLED)
        cv2.putText(img,button.text,(x+8,y+40),cv2.FONT_HERSHEY_PLAIN,3,(255,255,255),3)
        cvzone.cornerRect(img, (x,y, w,h),20, rt=0)
    return img

#Help to browser given query
def search(string):
    if string !="":
        browser.get("https://www.google.com/")
        browser.find_element_by_class_name("gLFyf").send_keys(string)
        browser.find_element_by_xpath("/html/body/div[1]/div[3]/form/div[1]/div[1]/div[3]/center/input[1]").click()   
        
cam = cv2.VideoCapture(0)
cam.set(3,1200)
cam.set(4,720)
buttonList = []
for i in range(len(keys)):
            for x,key in enumerate(keys[i]):
                buttonList.append(Button([80*x+90,80*i+50],key))
finalText =""    
while True:
    _,img  = cam.read()
    if _:
        img = detector.findHands(img)
        lmList, bboxInfo = detector.findPosition(img)
        img = drawAll(img,buttonList)
            
        if lmList:
            for button in buttonList:
                x,y = button.pos
                w,h = button.size
                
                if x<lmList[8][0]<x+w and y<lmList[8][1]<y+h:
                    cv2.rectangle(img,button.pos,
                                  (x+w,y+h),(175,0,175),
                                  cv2.FILLED)
                    cv2.putText(img,button.text,
                                (x+8,y+40)
                                ,cv2.FONT_HERSHEY_PLAIN,
                               3,(255,255,255),3)
                    l , _,_ = detector.findDistance(8,12,img,draw=False)
                    ## When Clicked
                    if l<30:
                        cv2.rectangle(img,button.pos,
                                  (x+w,y+h),(255,0,0),
                                  cv2.FILLED)
                        cv2.putText(img,button.text,
                                    (x+8,y+40)
                                    ,cv2.FONT_HERSHEY_PLAIN,
                                   3,(255,255,255),3)
                        if button.text==">":
                            search(finalText)
                            finalText =""
                        elif button.text =="<":
                            finalText = finalText[:-1]
                        else:
                            finalText +=button.text
                        sleep(0.15)
                        
        cv2.rectangle(img,(50,350),
                      (900,420),(175,0,175),
                      cv2.FILLED)
        cv2.putText(img,finalText,
                    (60,405)
                    ,cv2.FONT_HERSHEY_PLAIN,
                   3,(255,255,255),3)
        cv2.imshow("AI Keyboard",img)
        if cv2.waitKey(1)==13:
            break
cam.release()
cv2.destroyAllWindows()